In [3]:
import pandas as pd
import numpy as np
import flask
import json as js
import pyproj as pp
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///Hurricane.sqlite', echo=True)
con = engine.connect()

2019-03-19 19:57:14,315 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-03-19 19:57:14,317 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 19:57:14,318 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-03-19 19:57:14,320 INFO sqlalchemy.engine.base.Engine ()


In [10]:
con.execute('''CREATE TABLE Hurricanes(
                ID integer not null primary key, 
                Name Varchar (100),
                Time integer not null,
                Event Varchar (100),
                Status Varchar (100),
                Latitude integer not null,
                Longitude integer not null,
                Wind integer not null,
                Pressure integer not null,
                ISODate Varchar (100),
                Location Varchar (100)
                )''')


2019-03-19 20:32:05,719 INFO sqlalchemy.engine.base.Engine CREATE TABLE Hurricanes(
                ID integer not null primary key, 
                Name Varchar (100),
                Time integer not null,
                Event Varchar (100),
                Status Varchar (100),
                Latitude integer not null,
                Longitude integer not null,
                Wind integer not null,
                Pressure integer not null,
                ISODate Varchar (100),
                Location Varchar (100)
                )
2019-03-19 20:32:05,721 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 20:32:05,780 INFO sqlalchemy.engine.base.Engine COMMIT


In [20]:
df = pd.read_csv('hurdat.csv', index_col=0)
df.to_sql('Hurricanes', engine, if_exists='replace', index=True, index_label="id")

2019-03-19 20:38:44,708 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Hurricanes")
2019-03-19 20:38:44,709 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 20:38:44,712 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Hurricanes")
2019-03-19 20:38:44,713 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 20:38:44,717 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-03-19 20:38:44,718 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 20:38:44,721 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Hurricanes")
2019-03-19 20:38:44,721 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 20:38:44,726 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Hurricanes' AND type = 'table'
2019-03-19 20:38:44,727 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 20:38:44,729 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Hurricanes")
2

,ID


In [21]:
data = pd.read_sql('Select * from Hurricanes',engine)


2019-03-19 20:38:50,469 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Select * from Hurricanes")
2019-03-19 20:38:50,470 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 20:38:50,474 INFO sqlalchemy.engine.base.Engine Select * from Hurricanes
2019-03-19 20:38:50,474 INFO sqlalchemy.engine.base.Engine ()


In [22]:
data

,id,Name,Date,Time,Event,Status,Latitude,Longitude,Wind,Pressure,ISODate,Location
0,EP011949,UNNAMED,19490611,0,,TS,20.2,-106.3,45,-999,1949-06-11T00:00:00,East Pacific
1,EP011949,UNNAMED,19490611,600,,TS,20.2,-106.4,45,-999,1949-06-11T06:00:00,East Pacific
2,EP011949,UNNAMED,19490611,1200,,TS,20.2,-106.7,45,-999,1949-06-11T12:00:00,East Pacific
3,EP011949,UNNAMED,19490611,1800,,TS,20.3,-107.7,45,-999,1949-06-11T18:00:00,East Pacific
4,EP011949,UNNAMED,19490612,0,,TS,20.4,-108.6,45,-999,1949-06-12T00:00:00,East Pacific
5,EP011949,UNNAMED,19490612,600,,TS,20.5,-109.4,45,-999,1949-06-12T06:00:00,East Pacific
6,EP011949,UNNAMED,19490612,1200,,TS,20.6,-110.2,45,-999,1949-06-12T12:00:00,East Pacific
7,EP021949,UNNAMED,19490617,1200,,TS,14.1,-107.0,45,-999,1949-06-17T12:00:00,East Pacific
8,EP021949,UNNAMED,19490617,1800,,TS,14.6,-107.7,45,-999,1949-06-17T18:00:00,East Pacific
9,EP021949,UNNAMED,19490618,0,,TS,15.0,-108.4,45,-999,1949-06-18T00:00:00,East Pacific
